In [2]:
import pandas as pd
from dotenv import load_dotenv
import os
import requests
import json
import numpy as np

In [10]:
load_dotenv()

token1 = os.getenv('TOKEN1')
token2 = os.getenv('TOKEN2')
token_atual = token1

df = pd.read_csv("../dataset/dataset_filtrado.csv", delimiter=';')

api_call = 1

def get_paginated_data(url):
    global api_call
    global token_atual
    headers = {
        "Accept": "application/vnd.github+json",
        "Authorization": f"Bearer {token_atual}",
        "X-GitHub-Api-Version": "2022-11-28"
    }

    data = []
    if api_call == 4998:
        token_atual = token2
    while url:
        print(api_call, end=", ")
        response = requests.get(url, headers=headers)

        if response.status_code == 200:
            response_data = response.json()
            data.extend(parse_data(response_data))

            link_header = response.headers.get('Link')
            if link_header and 'rel="next"' in link_header:
                next_url = [url_part.split(";")[0].strip().strip("<>")
                            for url_part in link_header.split(",")
                            if 'rel="next"' in url_part]
                url = next_url[0] if next_url else None
            else:
                url = None
        else:
            print(f"Erro: {response.status_code}, {response.text}")
            break
        api_call = api_call + 1
    return data


def parse_data(response_data):
    if isinstance(response_data, list):
        return response_data
    elif isinstance(response_data, dict):
        namespace_key = list(response_data.keys())[0]
        return response_data[namespace_key]
    return []


def save_data_to_json(data, filename="data.json"):
    with open(filename, "w") as json_file:
        json.dump(data, json_file, indent=4)

In [ ]:
for url in df['URL']:

    parts = url.split('https://github.com/')[1].split('/')

    owner = parts[0]
    repo = parts[1]

    url_final = f"https://api.github.com/repos/{owner}/{repo}/issues?state=closed"

    print(url_final)

    data = get_paginated_data(url_final)

    filename = f"../data/issues/closed_issues_{owner}_{repo}.json"

    save_data_to_json(data, filename)    

    print() # apenas para formatar a saida

In [1]:
# Scpraing commits

urlApi = "https://api.github.com/repos"

urlRepo = df['URL'][0]

parts = urlRepo.split('https://github.com/')[1].split('/')

owner = parts[0]
repo = parts[1]

url = f"https://api.github.com/repos/{owner}/{repo}/commits"
headers = {
    "Accept": "application/vnd.github+json",
    "Authorization": f"Bearer {token}",
    "X-GitHub-Api-Version": "2022-11-28"
}

data = get_paginated_data(url, headers)

In [21]:
df

,URL,Unnamed: 0,Identifier,_id,FileInfo,NumAuthors,MonNauth,NumActiveMon,EarliestCommitDate,EarliestCommitDateConverted,...,gates,monitors,num_discos,shared_dbs,num_dockers,dockers_raw,structure_raw,ms_depend_graph,avg_size_service,commit_hash
0,https://github.com/mirumee/saleor,17,mirumee/saleor,ObjectId(62901b368b2efbc6e32df464),"{""Ruby"":17,""Perl"":291,""Rust"":1307,""Go"":1,""Sql""...",988.0,"{""2016-01"":12,""2020-07"":48,""2018-12"":46,""2016-...",103,1360687599,12.2.2013,...,[],['jaeger'],2,True,2,"{files:[{'path': '/requirements.txt', 'dbs': [...","{'path': '/.devcontainer/docker-compose.yml', ...","{'nodes': 4, 'edges': 0, 'avg_deps_per_service...",23336.000000,99e4b88c3e5545a80f784217e35d6b37cba13611
1,https://github.com/cfpb/hmda-platform,18,cfpb/hmda-platform,ObjectId(629087d38b2efbc6e3a1e16a),"{""Ruby"":5,""Perl"":1,""Scala"":4595,""TypeScript"":3...",300.0,"{""2016-01"":34,""2020-07"":52,""2018-12"":31,""2016-...",104,1361235364,19.2.2013,...,[],[],1,True,7,{files:[{'path': '/kubernetes/keycloak/auth/po...,"{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 3, 'edges': 0, 'avg_deps_per_service...",10849.000000,fd7836428253a19d5ca8b7d24a1757305f12efbb
2,https://github.com/golastmile/rasa_nlu,26,golastmile/rasa_nlu,ObjectId(62904e328b2efbc6e3e9d626),"{""Rust"":616,""Python"":2331,""Perl"":3,""other"":551...",867.0,"{""2021-07"":2,""2017-08"":22,""2020-07"":77,""2018-1...",62,1476448069,14.10.2016,...,[],[],0,False,2,"{files:[{'path': '/docs/package.json', 'dbs': ...","{'path': '/docker/docker-compose.yml', 'num_se...","{'nodes': 3, 'edges': 0, 'avg_deps_per_service...",15759.000000,221c8d36c77d6fccc7bcdfbbf61727f389501bde
3,https://github.com/samvera/hyrax,29,samvera/hyrax,ObjectId(6290354a8b2efbc6e3903ae5),"{""Ruby"":12962,""Perl"":188,""Python"":9,""PHP"":9,""o...",288.0,"{""2010-09"":5,""2016-01"":22,""2020-07"":13,""2018-1...",141,1269195778,21.3.2010,...,[],[],0,True,1,"{files:[{'path': '/package.json', 'dbs': [], '...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 6, 'edges': 7, 'avg_deps_per_service...",16399.000000,ed2b4fbb48dd61da78e93c741b86e2f04422b1cd
4,https://github.com/mesosphere/marathon,51,mesosphere/marathon,ObjectId(629090418b2efbc6e3ea1440),"{""Ruby"":1913,""Scala"":2488,""Clojure"":20,""Python...",488.0,"{""2016-01"":33,""2020-07"":4,""2018-12"":23,""2016-1...",98,1372172828,25.6.2013,...,[],[],1,True,10,{files:[{'path': '/tests/shakedown/requirement...,{'path': '/tests/performance/config/docker-com...,"{'nodes': 3, 'edges': 2, 'avg_deps_per_service...",5157.300000,90619da70d79a2ecf19368a21c6a695ea981bd74
5,https://github.com/openstates/openstates-scrapers,87,openstates/openstates-scrapers,ObjectId(629066908b2efbc6e33e6145),"{""Ruby"":18,""Clojure"":2,""Rust"":60,""Python"":4261...",272.0,"{""2010-09"":4,""2016-01"":9,""2020-07"":5,""2018-12""...",147,1235688657,26.2.2009,...,[],[],0,False,1,"{files:[], structure:{'path': '/docker-compose...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 4, 'edges': 0, 'avg_deps_per_service...",2875.000000,d528a6d1abb467ea7e07d53fb91dea2d947e2d77
6,https://github.com/letsencrypt/boulder,117,letsencrypt/boulder,ObjectId(629037588b2efbc6e3ab6e5e),"{""Perl"":19,""Python"":133,""Go"":6491,""other"":3312...",182.0,"{""2016-01"":24,""2020-07"":12,""2018-12"":7,""2016-1...",83,1419121794,21.12.2014,...,[],[],1,True,4,{files:[{'path': '/test/load-generator/require...,"{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 4, 'edges': 2, 'avg_deps_per_service...",8564.500000,824417f6c066e36d9048120db5369fd70b1fa92f
7,https://github.com/deviantony/docker-elk,172,deviantony/docker-elk,ObjectId(62902ab18b2efbc6e305af9d),"{""Ruby"":24,""Perl"":25,""TypeScript"":25,""Rust"":28...",1422.0,"{""2016-01"":19,""2020-07"":23,""2018-12"":49,""2016-...",86,1379881666,22.9.2013,...,[],[],0,False,11,"{files:[], structure:{'path': '/docker-compose...","{'path': '/docker-compose.yml', 'num_services'...","{'nodes': 4, 'edges': 3, 'avg